In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib.pyplot as plt #Data Visualization 
import seaborn as sns  #Python library for Vidualization

In [ ]:
data = pd.read_csv('/kaggle/input/odb-paris/ParisHousing.csv', index_col = 0) #применять в случае если 1 столбец пустой

In [ ]:
data.head()

In [ ]:
data.shape, data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.columns

# EDA

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
plt.figure(1 , figsize = (15 , 5))
sns.countplot(y = 'category' , data = data)
plt.show()

In [ ]:
plt.figure(1 , figsize = (15 , 6))
n = 0 
for x in ['squareMeters' , 'price' , 'floors']:
    n += 1
    plt.subplot(1 , 3 , n)
    plt.subplots_adjust(hspace =0.5 , wspace = 0.5)
    sns.distplot(data[x] , bins = 20)
    plt.title('Distplot of {}'.format(x))
plt.show()

In [ ]:
sns.set_style('darkgrid')
g = sns.FacetGrid(data,hue="category",palette='coolwarm',size=6,aspect=2)
g = g.map(plt.hist,'price',bins=20,alpha=0.7)

# Prepare dataset

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['numberOfRooms'] = label_encoder.fit_transform(data["numberOfRooms"])
data['category'] = label_encoder.fit_transform(data["category"])
data['PoolAndYard'] = label_encoder.fit_transform(data["PoolAndYard"])

In [ ]:
data[['hasStormProtector', 'isNewBuilt', 'hasStorageRoom']] = data[['hasStormProtector', 
                                                                 'isNewBuilt', 'hasStorageRoom']].astype(int)

In [ ]:
data.head()

In [ ]:
data.describe()

# Стандартизация данных

Поскольку алгоритм K-ближайших соседей делает прогнозы относительно точки данных (семпла), используя наиболее близкие к ней наблюдения, существующий масштаб показателей в датасете имеет большое значение.

Из-за этого специалисты по машинному обучению обычно стандартизируют набор данных, что означает корректировку каждого значения x так, чтобы они находились примерно в одном диапазоне.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(data)
scaled_features = scaler.transform(data)

В качестве результата мы получили массив NumPy со всеми точками данных из датасета, но нам желательно преобразовать его в формат DataFrame библиотеки pandas

In [ ]:
scaled_data = pd.DataFrame(scaled_features, columns = data.columns)

In [ ]:
scaled_data.head()

# Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

In [ ]:
wcss=[]
for i in range(1,100, 20):
    kmeans = KMeans(n_clusters= i, init='k-means++', random_state=0)
    kmeans.fit(scaled_data)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(range(1,100, 20), wcss)
plt.title('Метод локтя')
plt.xlabel('Количество кластеров')
plt.ylabel('wcss')
plt.show()

In [ ]:
from itertools import combinations

In [ ]:
lst_vars=list(combinations(scaled_data.columns,2))

In [ ]:
len(lst_vars)